In [3]:
#extracting images from xlsx
from openpyxl import load_workbook
from PIL import Image
import io
import pandas as pd
import os

dataset_path = 'HackUMass Data.xlsx'
img_directory_path = 'ML Pipeline/extracted_images'
updated_path = '/Users/alishasrivastava/Desktop/ML Pipeline/updated.csv'

wb = load_workbook('HackUMass Data.xlsx')['Sheet1']
for x, img in enumerate(wb._images):
    img = Image.open(io.BytesIO(img._data()))
    img.save(f'extracted_images/img_{x}.png') #TODO: is png best for MongoDB?

df = pd.read_excel(dataset_path)
img_paths = [os.path.join(img_directory_path, f'img_{x}') for x in range(len(df))]
df['img_paths'] = img_paths
df.to_csv(updated_path, index = False)


FileNotFoundError: [Errno 2] No such file or directory: 'extracted_images/img_0.png'

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os

In [4]:
data = pd.read_csv('/Users/alishasrivastava/Desktop/ML Pipeline/updated.csv')
image_folder_path = '/Users/alishasrivastava/Desktop/ML Pipeline/extracted_images'
data.head()


,Images,Category,subCategory,articleType,Color,Season,Usage,Gender,img_paths
0,NaN,Apparel,Tops,Strapless Top,Maroon,Summer,Casual,Female,/Users/alishasrivastava/Desktop/ML Pipeline/ex...
1,NaN,Apparel,Outerwear,Puffer Jacket,White,Winter,Casual,Unisex,/Users/alishasrivastava/Desktop/ML Pipeline/ex...
2,NaN,Apparel,Dresses,Mini Dress,Burgundy,Winter,Casual,Female,/Users/alishasrivastava/Desktop/ML Pipeline/ex...
3,NaN,Apparel,Sweaters,Sweater,Maroon,Winter,Casual,Unisex,/Users/alishasrivastava/Desktop/ML Pipeline/ex...
4,NaN,Apparel,Sweaters,Cardigan,Grey,Winter,Casual,Unisex,/Users/alishasrivastava/Desktop/ML Pipeline/ex...


In [15]:
image_column = 'img_paths'
target_columns = ['Category', 'subCategory', 'articleType', 'Color', 'Season', 'Usage', 'Gender']  # target columns
encoded_targets = {}
for col in target_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    encoded_targets[col] = le  # store the encoder for each column if needed for later decoding
data['img_paths'] = data['img_paths'].apply(lambda x: f"{x}.png")

In [16]:
X = data[image_column]
y = data[target_columns]
y = pd.get_dummies(y, columns=target_columns)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
print(data[image_column].head())

0    /Users/alishasrivastava/Desktop/ML Pipeline/ex...
1    /Users/alishasrivastava/Desktop/ML Pipeline/ex...
2    /Users/alishasrivastava/Desktop/ML Pipeline/ex...
3    /Users/alishasrivastava/Desktop/ML Pipeline/ex...
4    /Users/alishasrivastava/Desktop/ML Pipeline/ex...
Name: img_paths, dtype: object


In [17]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.concat([X_train, y_train], axis=1),
    directory=image_folder_path,
    x_col=image_column,
    y_col=y.columns.tolist(),  # Updated to handle all encoded columns
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'  # For multi-output with one-hot encoding
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.concat([X_test, y_test], axis=1),
    directory=image_folder_path,
    x_col=image_column,
    y_col=y.columns.tolist(),
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw'
)

Found 34 validated image filenames.
Found 9 validated image filenames.


In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')  # single softmax output per encoded category
])

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 598ms/step - accuracy: 0.7206 - loss: 4641.7588 - val_accuracy: 0.0000e+00 - val_loss: 10385.0107
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.0000e+00 - loss: 9759.2578 - val_accuracy: 0.7778 - val_loss: 20260.8359
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - accuracy: 0.8002 - loss: 18747.7500 - val_accuracy: 0.0000e+00 - val_loss: 35537.2070
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - accuracy: 0.0000e+00 - loss: 32414.4863 - val_accuracy: 0.0000e+00 - val_loss: 62233.1406
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.0000e+00 - loss: 56650.9492 - val_accuracy: 0.0000e+00 - val_loss: 102320.5859
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 697ms/step - accuracy: 0.0392 - loss: 115153.1250 - val_accuracy: 0.7778 - val_loss: 166554.3438
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 0.8002 - loss: 148551.7188 - val_accuracy: 0.0000e+00 - val_loss: 260279.3594
Epoch 8/10
2/2 ━━━━━━━━━━━━

In [24]:
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.7778 - loss: 871316.0625
Test Accuracy: 0.7777777910232544


In [ ]:
#trying one test case

df = pd.read_csv('/Users/alishasrivastava/Desktop/ML Pipeline/updated.csv')
row = df.iloc[0]
path = row['img_paths']
path = f'{path}.png'
img = Image.open(path)
if img.mode == 'RGBA':
    img = img.convert('RGB')

img = img.resize((224, 224))
img_array = np.array(img)
img_array = img_array / 255.0 
img_array = np.expand_dims(img_array, axis=0)

#predict
prediction = model.predict(img_array)
target_columns = ['Category', 'subCategory', 'articleType', 'Color', 'Season', 'Usage', 'Gender']
predicted_labels = {}
for i, col in enumerate(target_columns):
    predicted_class_index = np.argmax(prediction[0][i])
    predicted_labels[col] = df[col].unique()[predicted_class_index]

print(f"Predicted Labels: {predicted_labels}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predicted Labels: {'Category': 'Apparel', 'subCategory': 'Tops', 'articleType': 'Strapless Top', 'Color': 'Maroon', 'Season': 'Summer', 'Usage': 'Casual', 'Gender': 'Female'}


In [41]:
model.save('model.keras')